# Cancer Patient  
- Multiple office visit notes are read
- Compiled in a string (no particular chronological order)
- chatGPT parses and list chronologically with
    - Clinical Findings
    - Medications
    - Lab Values

In [2]:
import os
import openai
import webbrowser
import datetime 

from chatGPT_utilities import setup_OpenAI_API
from chatGPT_utilities import json_to_dict
from chatGPT_utilities import get_completion
from chatGPT_utilities import get_completion_from_messages_FULL
from chatGPT_utilities import read_file
from chatGPT_utilities import token_usage
from chatGPT_utilities import print_token_usage

openai.api_key = setup_OpenAI_API()
print(f'Current Directory:\n{os.getcwd()}')

starting_directory /Users/michael/Coding
Directory:/Users/michael/Coding
Current Directory:
/Users/michael/Coding


# Read back the compilation of notes  

- Notes are not in chronological order

In [7]:


def read_patient_notes():
    Cancer_Patient_Notes_file = 'Cancer_Patient_Notes'
    os.chdir(Cancer_Patient_Notes_file)

    list_of_notes = [i for i in os.listdir() if i.endswith('txt') and '_20' in i]

    compilation_of_notes = ''
    print(f'Processing')
    for num, name in enumerate(list_of_notes, start=1): 
        print("{0:3}: {1}".format(num, name))
    print('Note : The files are not necessarily in chronological order')

    for file_name in list_of_notes:

    # for date in ['Aug_20']:
        print(f'\033[91m                                     Date of Note: \033[1m{file_name}\033[0m') 
        interim_note = read_file(file_name)
        print(interim_note)
        compilation_of_notes= compilation_of_notes + '  \n\n' + interim_note
    print('\n\n\n\n\033[91m'+ '\033[1m--------------------------COMPILATION of NOTES---------\033[0m') 
    print(compilation_of_notes)


    os.chdir('..')

    if os.getcwd() != starting_directory:
        os.chdir(starting_directory)
    assert os.getcwd() == starting_directory , 'We should be in starting directory'
    return compilation_of_notes

In [8]:
starting_directory = os.getcwd()
compilation_of_notes = read_patient_notes()

Processing
  1: Nov_20.txt
  2: Oct_20.txt
  3: Sept_20.txt
  4: Aug_20.txt
Note : The files are not necessarily in chronological order
                                     Date of Note: Nov_20.txt
Date: November 20


Following the completion of induction chemotherapy, he is experiencing a mix of physical and emotional symptoms. Today his ANC 900, HCT 30

Physically, he is still recovering from the side effects of chemotherapy. Reports fatigue, weakness, and a general sense of low energy. Nausea and vomiting gradually subsiding. Has reduced appetite, and has lost 2 kg since initial diagnosis. Emotionally, the patient feeling some uncertainty as awaiting the results of the treatment response evaluation and bone marrow assessment. Will continue to provide supportive care, address any ongoing symptoms, and offer emotional support prn.


                                     Date of Note: Oct_20.txt
Date: Oct 20
This 65-year-old white male was 
diagnosed with acute myeloid leukemia (AML) in

In [9]:
prompt_1 = f"""
Your task is to extract relevant information from 
clinical note. 

From the compilation of notes below, delimited by triple quotes \
extract the information relevant to patient's condition. 

Patient's Diagnosis:
Date of Diagnosis:


List the dates of visits in chronological order and 
include clinical findings, lab values and any medications mentioned during that visit
 in the following format:

Date # 1 - 
    Clinical Findings    
    Medications:
    
Date # 2 - 
    Clinical Findings   
    Medications:…
Date # N - 
    Clinical Findings
    Medications:


Review: ```{compilation_of_notes}```
"""

In [10]:
print('\n\n\n\n\033[91m'+ '\033[1mSTART of QUERY\033[0m') 
start_time = datetime.datetime.now() 
print('start_time:',  start_time.strftime("%H:%M:%S"))
messages = [{"role": "user", "content": prompt_1}]
response_details = get_completion_from_messages_FULL(messages)
response = response_details.choices[0].message["content"]
    # print(response)

# response = get_completion(prompt)


end_time = datetime.datetime.now() 
print(response, '\n'*2)
print('end_time  :', end_time.strftime("%H:%M:%S"))
elapsed_time = end_time - start_time
print('Elapsed_time: ',elapsed_time.seconds,'.',int(elapsed_time.microseconds/1000),' seconds',sep='')
cost = print_token_usage(response_details)
print(f'Cost to run query: ${cost}')





START of QUERY
start_time: 15:06:46
Patient's Diagnosis: Acute Myeloid Leukemia (AML)
Date of Diagnosis: August 2020

Date: August 20
Clinical Findings: Fatigue and lethargy
Medications: Statin and finasteride
Lab Values: ANC 1500, HCT 27

Date: Sept 20
Clinical Findings: Completed induction chemotherapy, achieved complete remission
Medications: None mentioned
Lab Values: None mentioned

Date: Oct 20
Clinical Findings: Undergoing consolidation therapy, experiencing common side effects of chemotherapy such as fatigue, lethargy, nausea, vomiting, decreased appetite, and weight loss
Medications: Arabinosylcytosine and midostaurin
Lab Values: ANC 1250, HCT 29

Date: November 20
Clinical Findings: Experiencing physical and emotional symptoms such as fatigue, weakness, low energy, nausea, vomiting, reduced appetite, and weight loss
Medications: None mentioned
Lab Values: ANC 900, HCT 30 


end_time  : 15:06:55
Elapsed_time: 9.644 seconds
Cost to run query is based on token usage, which i

In [11]:
print('\n\n\n\n\033[91m'+ '\033[1mSTART of QUERY\033[0m') 
start_time = datetime.datetime.now() 
print('start_time:',  start_time.strftime("%H:%M:%S"))


prompt_2 = f"""
Your task is to extract relevant information from clinical note. 

From the compilation of notes below, delimited by triple quotes \
extract the information relevant to patient's condition. 

Patient's Diagnosis:
Date of Diagnosis:


List the dates of visits in chronological order and include the lab values in the following format
Date # 1 : 
   HCT :
   ANC : 
    
Date # 2 : 
   HCT :
   ANC : 

Date # N :
   HCT :
   ANC : 

Do not list any additional information in the synopsis other than lab values.
If no lab values are listed for a given date, indicate 'N/A'

Review: ```{compilation_of_notes}```
"""


messages = [{"role": "user", "content": prompt_2}]
response_details = get_completion_from_messages_FULL(messages)
response = response_details.choices[0].message["content"]
    # print(response)

# response = get_completion(prompt)
end_time = datetime.datetime.now() 
print(response, '\n'*2)
print('end_time  :', end_time.strftime("%H:%M:%S"))
elapsed_time = end_time - start_time
print('Elapsed_time: ',elapsed_time.seconds,'.',int(elapsed_time.microseconds/1000),' seconds',sep='')
cost = print_token_usage(response_details)
print(f'Cost to run query: ${cost}')





START of QUERY
start_time: 15:07:26
Patient's Diagnosis: Acute Myeloid Leukemia (AML)
Date of Diagnosis: August 2020

Dates of visits in chronological order with lab values:

August 20:
   HCT: 27
   ANC: 1500

September 20:
   N/A

October 20:
   HCT: 29
   ANC: 1250

November 20:
   HCT: 30
   ANC: 900 


end_time  : 15:07:32
Elapsed_time: 6.487 seconds
Cost to run query is based on token usage, which is $0.002 / 1K tokens


Token Usage:


completion_tokens: 94
prompt_tokens    : 676
total_tokens     : 770
Cost to run query: $0.002


# Assemble Prompts in a List of Prompts

In [12]:
prompt_list = [prompt_1,prompt_2]

In [13]:
for prompt in prompt_list:
    start_time =   datetime.datetime.now() 
    relevant_details = prompt.split("\n\nReview:")[0]
    print(f'\033[91m\033[1m {relevant_details}\033[0m') 
    messages = [{"role": "user", "content": prompt}]
    response_details = get_completion_from_messages_FULL(messages)
    response = response_details.choices[0].message["content"]
        # print(response)

    # response = get_completion(prompt)
    end_time = datetime.datetime.now() 
    
    end_time = datetime.datetime.now() 
    print(response, '\n'*2)
    print('end_time  :', end_time.strftime("%H:%M:%S"))
    elapsed_time = end_time - start_time
    print('Elapsed_time: ',elapsed_time.seconds,'.',int(elapsed_time.microseconds/1000),' seconds',sep='')
    cost = print_token_usage(response_details)
    print(f'Cost to run query: ${cost}\n\n') 
    

 
Your task is to extract relevant information from 
clinical note. 

From the compilation of notes below, delimited by triple quotes extract the information relevant to patient's condition. 

Patient's Diagnosis:
Date of Diagnosis:


List the dates of visits in chronological order and 
include clinical findings, lab values and any medications mentioned during that visit
 in the following format:

Date # 1 - 
    Clinical Findings    
    Medications:
    
Date # 2 - 
    Clinical Findings   
    Medications:…
Date # N - 
    Clinical Findings
    Medications:

Patient's Diagnosis: Acute Myeloid Leukemia (AML)
Date of Diagnosis: August 2020

Date: August 20
Clinical Findings: Fatigue, lethargy
Medications: Statin, finasteride
Lab Values: ANC 1500, HCT 27

Date: Sept 20
Clinical Findings: Completed induction chemotherapy, achieved complete remission
Medications: N/A
Lab Values: N/A

Date: Oct 20
Clinical Findings: Undergoing consolidation therapy, experiencing fatigue, lethargy, nausea,